In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from os import listdir
from os.path import isfile, join, isdir
from math import hypot

%matplotlib inline

Methods for querying. top_n_crops_produced_at_point is broken

In [2]:
def find_value_for_point(x, y, df):
    closest_x = df.iloc[(np.abs(df['x'] - x)).idxmin()]['x']
    closest_y = df.iloc[(np.abs(df['y'] - y)).idxmin()]['y']
    return df[(df['x'] == closest_x) & (df['y'] == closest_y)]

In [3]:
# Returns a list of the n most produced crops for the given point.
# Returns it as a list of n triples, for each triple:
# - First item is a tuple containing the name of the crop.
# - Seond item is a float value representing the production of the above crop at given point.

def top_n_crops_produced_at_point(x, y, n, df):
    # Get the cell corresponding to the point. In the Data, x and y are flipped so flip'em
    cell = find_value_for_point(y, x, df)
    if (cell.empty):
        return None
    # Ignore values for column, row, x, y for the query.
    cell = cell[cell.columns.difference(['x', 'y'])]
    best_crops = cell.sort_values(by=cell.index[0], ascending=False, axis=1).iloc[:,0:n]
    top_n = []   
    for column in best_crops:
        crop = best_crops[column]
        crop_name = crop.name
        value = round(crop.values[0], 3)
        if (value != 0):
            top_n.append((crop_name, value))
    return top_n

In [4]:
# Returns a list of the n most productive cells for the given crop.
# Returns it as a list of n triples, for each triple:
# - First item is a tuple containing the Latitude/Longitude of the point.
# - Seond item is a float value representing the production of the input crop at above point.

def top_n_production_points_for_crop(crop_name, n, df):
    best_cells = df.loc[df[crop_name].nlargest(n).index]
    top_n = []
    for index, cell in best_cells.iterrows():
        # Round coordinates to match Lat/Long format
        x = round(cell['x'], 6)
        y = round(cell['y'], 6)
        value = round(cell[crop_name], 4)
        bundle = ((y, x), value)
        top_n.append(bundle)
    return top_n

## Trying to merge tables for banana production and precipitation

In [19]:
df_banana = pd.read_csv('./bananas.csv')

In [20]:
df_precp = pd.read_csv('./output_clim.csv')

In [21]:
df_banana = df_banana.rename(columns={'Z': 'BANA'})
df_precp = df_precp.rename(columns={'Z': 'precp'})

In [22]:
df_banana['X'] = round(df_banana['X'], 6)
df_banana['Y'] = round(df_banana['Y'], 6)

df_precp['X'] = round(df_precp['X'], 6)
df_precp['Y'] = round(df_precp['Y'], 6)

In [24]:
print(len(df_precp['X']))
print(len(df_banana['X']))

9331200
8004960


In [25]:
merged = pd.concat([df_precp, df_banana['BANA']], axis=1)

In [26]:
print(len(merged['X']))

9331200


In [28]:
top_n_production_points_for_crop('BANA', 2, merged)

[((10.375, 78.291667000000004), 299639.0),
 ((10.541667, 78.208332999999996), 273557.8125)]

In [31]:
top_n_crops_produced_at_point(10.375, 78.291667, 1, merged)

[('BANA', 299639.0)]

## Get aggregated data

In [5]:
df_spam = pd.read_csv('../data/my_aggregates/rain_fed_production.csv')

In [6]:
df_spam

x          y         01        02         03           04  \
0        -37.166667  83.666667   0.000000  0.000000  17.645822     0.000000   
1        -37.083333  83.666667   0.000000  0.000000  19.059517     0.000000   
2        -37.000000  83.666667   0.000000  0.000000  20.045350     0.000000   
3        -36.916667  83.666667   0.000000  0.000000  19.985059     0.000000   
4        -36.833333  83.666667   0.000000  0.000000  19.797453     0.000000   
5        -36.750000  83.666667 -12.413750  4.895833  19.895068   871.897990   
6        -36.666667  83.666667 -16.252917  7.055833  19.896536  1150.757562   
7        -36.583333  83.666667 -15.886875  7.403750  19.887874  1134.993392   
8        -36.500000  83.666667 -15.843750  7.357500  20.580820  1129.706780   
9        -36.416667  83.666667 -15.953958  7.306250  19.602653  1134.549364   
10       -36.333333  83.666667 -15.827708  7.315416  19.153922  1131.867685   
11       -36.250000  83.666667 -15.903750  7.371667  19.401286  1142.249180   
12       -36.166667  83.666667 -16.033929  7.475000  19.451932  1159.047580   
13       -35.833333  83.666667   0.000000  0.000000  19.783465     0.000000   
14       -35.750000  83.666667 -10.900214  5.302992  19.560451   787.229333   
15       -35.666667  83.666667  -9.677583  4.481167  19.246032   702.851693   
16       -35.583333  83.666667  -9.625167  4.333000  18.849101   699.653008   
17       -35.500000  83.666667 -11.727744  5.299390  18.903915   847.425323   
18       -35.416667  83.666667  -1.200625  0.541250  19.587628    86.526262   
19       -35.333333  83.666667  -4.714881  2.153571  19.885914   334.259792   
20       -35.250000  83.666667 -12.990833  5.948333  19.102844   938.223601   
21       -35.166667  83.666667 -14.550390  6.592969  18.725860  1061.402751   
22       -35.083333  83.666667 -15.871389  7.106111  18.644482  1171.967238   
23       -35.000000  83.666667 -15.932292  7.164584  18.879899  1177.299410   
24       -34.750000  83.666667 -16.605556  7.600000  18.675801  1204.233145   
25       -34.666667  83.666667 -10.786759  5.016111  18.919273   779.649510   
26       -34.583333  83.666667  -8.204025  3.720198  19.107589   604.190856   
27       -34.500000  83.666667  -8.028542  3.555417  19.286491   590.215045   
28       -34.416667  83.666667  -6.907548  3.040942  19.451993   507.684668   
29       -34.333333  83.666667  -4.093155  1.816786  18.887190   298.450280   
...             ...        ...        ...       ...        ...          ...   
3170184  177.500000 -89.916667 -24.783333  3.395000        NaN   593.652730   
3170185  177.583333 -89.916667 -24.783916  3.399500        NaN   593.818401   
3170186  177.666667 -89.916667 -24.783583  3.405500        NaN   593.806872   
3170187  177.750000 -89.916667 -24.784958  3.408250        NaN   593.857553   
3170188  177.833333 -89.916667 -24.787000  3.409500        NaN   593.970530   
3170189  177.916667 -89.916667 -24.788500  3.410333        NaN   594.104637   
3170190  178.000000 -89.916667 -24.790000  3.409833        NaN   594.252241   
3170191  178.083333 -89.916667 -24.790916  3.409000        NaN   594.475841   
3170192  178.166667 -89.916667 -24.791125  3.407583        NaN   594.673134   
3170193  178.250000 -89.916667 -24.791375  3.409083        NaN   594.893523   
3170194  178.333333 -89.916667 -24.790750  3.411167        NaN   595.036153   
3170195  178.416667 -89.916667 -24.790375  3.412250        NaN   595.141318   
3170196  178.500000 -89.916667 -24.790625  3.413084        NaN   595.204218   
3170197  178.583333 -89.916667 -24.791708  3.411250        NaN   595.357747   
3170198  178.666667 -89.916667 -24.793000  3.411500        NaN   595.511145   
3170199  178.750000 -89.916667 -24.793167  3.411666        NaN   595.654117   
3170200  178.833333 -89.916667 -24.793625  3.412250        NaN   595.881197   
3170201  178.916667 -89.916667 -24.794333  3.417166        NaN   596.129752   
3170202  179.000000 -89.916667 -24.795542  3.420417        NaN   596

In [8]:
top_n_production_points_for_crop('MAIZ', 2, df_spam)

[((20.416667, -103.083333), 66097.0), ((20.416667, -102.583333), 63475.5)]

Query the most producing spot for bananas

In [46]:
top_n_crops_produced_at_point(-29.000221, -59.0255638, 3, df_spam)

[('VEGE', 104.9), ('RICE', 57.1), ('SWPO', 43.9)]

In [37]:
merged.loc[merged['BANA'].nlargest(1).index]

X       Y  precp      BANA
4128699  78.291667  10.375  968.0  299639.0

In [8]:
df_soil = pd.read_csv('./AWCh1_M_sl4_10km_ll.csv')

In [13]:
top_n_production_points_for_crop('Z', 9, df_soil[df_soil['Z'] != 255])

[((-3.8508330000000002, 115.55), 32.0),
 ((73.349166999999994, 128.55000000000001), 31.0),
 ((73.349166999999994, 128.44999999999999), 30.0),
 ((72.949167000000003, 129.15000000000001), 30.0),
 ((68.949167000000003, -26.949999999999999), 30.0),
 ((2.849167, 111.34999999999999), 30.0),
 ((73.549166999999997, 140.65000000000001), 29.0),
 ((73.449167000000003, 127.55), 29.0),
 ((73.449167000000003, 128.05000000000001), 29.0)]

In [14]:
df_clim = pd.read_csv('./world_clim_bio_vars.csv')

In [23]:
# top_n_production_points_for_crop('01', 12, df_clim)
find_value_for_point(41.208333000000003, 12.958333, df_clim)['01']

3994334    33.661583
Name: 01, dtype: float64

In [7]:
df_merged = pd.read_csv('/Volumes/Christian_2015/project-eden/my_aggregates/production_clim_merged_georasters.csv')

In [91]:
# top_n_production_points_for_crop('bana', 1, df_merged)
pt = (37.069624, -120.411867)
# top_n_production_points_for_crop('12', 1, df_merged)
m = find_value_for_point(pt[1], pt[0], df_merged)

In [58]:
# top_n_crops_produced_at_point(10.541667, 78.208332999999996, 1, df_merged)
# df_merged[]

In [9]:
df_spam_original = pd.read_csv('/Volumes/Christian_2015/project-eden/my_aggregates/rain_fed_production.csv')

In [10]:
df_spam_original

X          Y  SWPO  TOBA  SOYB  REST  OCER  TEMF  TEAS  \
0       -179.958333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1       -179.875000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2       -179.791667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3       -179.708333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4       -179.625000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5       -179.541667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
6       -179.458333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
7       -179.375000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8       -179.291667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
9       -179.208333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
10      -179.125000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
11      -179.041667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
12      -178.958333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
13      -178.875000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
14      -178.791667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
15      -178.708333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
16      -178.625000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
17      -178.541667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
18      -178.458333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
19      -178.375000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
20      -178.291667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
21      -178.208333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
22      -178.125000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
23      -178.041667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
24      -177.958333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
25      -177.875000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
26      -177.791667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
27      -177.708333  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28      -177.625000  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
29      -177.541667  89.958333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...             ...        ...   ...   ...   ...   ...   ...   ...   ...   
8004930  177.541667 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004931  177.625000 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004932  177.708333 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004933  177.791667 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004934  177.875000 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004935  177.958333 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004936  178.041667 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004937  178.125000 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004938  178.208333 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004939  178.291667 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004940  178.375000 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004941  178.458333 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004942  178.541667 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004943  178.625000 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004944  178.708333 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004945  178.791667 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004946  178.875000 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004947  178.958333 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004948  179.041667 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004949  179.125000 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8004950  179.208333 -64.375000   0.0   0.0   0.0   0.0   0.0   0.0

In [15]:
df_spam_original = df_spam_original.rename(columns={'X': 'x', 'Y': 'y'})

In [16]:
top_n_production_points_for_crop('BANA', 1, df_spam_original)

[((10.375, 78.291667000000004), 299639.0)]

In [11]:
df_spam_aggr = pd.read_csv('/Volumes/Christian_2015/project-eden/my_aggregates/spam_production_georasters.csv')

In [12]:
df_spam_aggr

x          y  acof  bana  barl  bean  cass  chic  cnut  coco  \
0       27.583333  71.166667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1       27.666667  71.166667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2       23.833333  71.083333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3       23.916667  71.083333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4       24.000000  71.083333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5       24.666667  71.083333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
6       24.750000  71.083333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
7       25.666667  71.083333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8       28.083333  71.083333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
9       23.916667  71.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
10      24.000000  71.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
11      25.666667  71.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
12      27.666667  71.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
13      23.333333  70.916667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
14      24.833333  70.916667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
15      25.666667  70.916667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
16      25.750000  70.916667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
17      26.666667  70.916667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
18      27.250000  70.916667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
19      29.166667  70.916667   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
20      23.250000  70.833333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
21      24.333333  70.833333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
22      24.416667  70.833333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
23      24.583333  70.833333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
24      25.166667  70.833333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
25      25.500000  70.833333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
26      26.583333  70.833333   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
27      22.333333  70.750000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28      22.833333  70.750000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
29      22.916667  70.750000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...           ...        ...   ...   ...   ...   ...   ...   ...   ...   ...   
747652 -69.416667 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747653 -69.333333 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747654 -69.250000 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747655 -69.166667 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747656 -69.083333 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747657 -69.000000 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747658 -68.916667 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747659 -68.833333 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747660 -68.750000 -55.083333   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747661 -69.666667 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747662 -69.583333 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747663 -69.333333 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747664 -69.083333 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747665 -68.916667 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747666 -68.750000 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747667 -68.583333 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747668 -68.500000 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747669 -68.416667 -55.166667   0.0   0.0  27.4   0.0   0.0   0.0   0.0   0.0   
747670 -68.33333

In [13]:
top_n_production_points_for_crop('bana', 1, df_spam_aggr)

[((10.416667, 78.25), 299639.0)]